# Introducción a PyArrow

**Apache Arrow** es un estándar moderno para el almacenamiento y transferencia eficiente de datos columnar. **PyArrow** es la implementación de Arrow en Python, que proporciona una interfaz de alto rendimiento para trabajar con datos estructurados.

En este notebook exploraremos:
* Conceptos fundamentales de Arrow
* Creación y manipulación de Arrays y Tables
* Integración con Pandas como backend
* Formatos soportados (Parquet, ORC, CSV)
* Comparativas de rendimiento y memoria

## ¿Qué es PyArrow?

### Historia y Contexto

Apache Arrow es un proyecto de código abierto que define un formato estandarizado para datos columnar en memoria. Fue diseñado para resolver problemas comunes en el ecosistema de datos:

1. **Heterogeneidad de formatos:** Cada librería tenía su propia representación interna (NumPy, Pandas, etc.)
2. **Ineficiencia de memoria:** Necesidad de copias repetidas entre sistemas
3. **Cuellos de botella en I/O:** Lectura/escritura lenta de formatos binarios

### Ventajas de Arrow

| Ventaja | Descripción |
| :--- | :--- |
| **Memoria eficiente** | Formato columnar zero-copy |
| **Interoperabilidad** | Compatible con múltiples lenguajes (C++, Java, Python, Rust) |
| **Velocidad** | Operaciones vectorizadas en GPU |
| **Compresión** | Soporta múltiples esquemas de compresión |
| **Estandarización** | Formato universal para datos complejos |

### PyArrow como Backend de Pandas

Desde **Pandas 2.0**, es posible usar PyArrow como backend para DataFrames, reemplazando el antiguo sistema de tipos NumPy con tipos más eficientes y flexibles.

In [ ]:
# Importar las librerías necesarias
import pyarrow as pa
import pandas as pd
import numpy as np
import sys

print(f"PyArrow versión: {pa.__version__}")
print(f"Pandas versión: {pd.__version__}")
print(f"NumPy versión: {np.__version__}")

## Arrays de PyArrow

Un Array de PyArrow es una colección homogénea de valores de un único tipo, almacenada eficientemente en memoria.

### Creación de Arrays

Se pueden crear arrays a partir de:
* Listas de Python
* Arrays de NumPy
* Iteradores

In [ ]:
# Crear array desde una lista Python
arr_simple = pa.array([1, 2, 3, 4, 5])
print(f"Array: {arr_simple}")
print(f"Tipo: {arr_simple.type}")
print(f"Longitud: {len(arr_simple)}")

In [ ]:
# Array con valores nulos
arr_nulls = pa.array([1, 2, None, 4, 5])
print(f"Array con nulos: {arr_nulls}")
print(f"Contiene nulos: {arr_nulls.null_count > 0}")

In [ ]:
# Tipos de datos explícitos
arr_int32 = pa.array([1, 2, 3], type=pa.int32())
arr_float64 = pa.array([1.5, 2.5, 3.5], type=pa.float64())
arr_string = pa.array(['apple', 'banana', 'cherry'], type=pa.string())

print(f"Int32: {arr_int32} - Tipo: {arr_int32.type}")
print(f"Float64: {arr_float64} - Tipo: {arr_float64.type}")
print(f"String: {arr_string} - Tipo: {arr_string.type}")

In [ ]:
# Conversión desde NumPy
np_array = np.array([10, 20, 30, 40, 50])
pa_array = pa.array(np_array)
print(f"NumPy → PyArrow: {pa_array}")
print(f"Tipo resultante: {pa_array.type}")

### Operaciones con Arrays

PyArrow proporciona operaciones vectorizadas similares a NumPy pero con mejor rendimiento:

In [ ]:
import pyarrow.compute as pc

arr = pa.array([1, 2, 3, 4, 5, 6])

# Operaciones aritméticas
print(f"Array original: {arr}")
print(f"Suma 10: {pc.add(arr, 10)}")
print(f"Multiplicar por 2: {pc.multiply(arr, 2)}")

# Funciones de agregación
print(f"\nSum: {pc.sum(arr).as_py()}")
print(f"Media: {pc.mean(arr).as_py()}")
print(f"Máximo: {pc.max(arr).as_py()}")
print(f"Mínimo: {pc.min(arr).as_py()}")

## Tables de PyArrow

Una **Table** es una colección de Arrays columnar con schema definido. Es el equivalente de un DataFrame.

In [ ]:
# Crear una Table desde diccionario
data = {
    'id': [1, 2, 3, 4, 5],
    'nombre': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'edad': [25, 30, 35, 28, 32],
    'salario': [50000.0, 60000.0, 75000.0, 55000.0, 70000.0]
}

table = pa.table(data)
print(f"Table:\n{table}")
print(f"\nSchema:\n{table.schema}")

In [ ]:
# Acceso a columnas
print(f"Columna 'nombre': {table['nombre']}")
print(f"\nDimensiones: {table.shape}")
print(f"Nombres de columnas: {table.column_names}")

In [ ]:
# Conversión a pandas
df_pandas = table.to_pandas()
print(f"DataFrame (Pandas):\n{df_pandas}")
print(f"\nDtypes:\n{df_pandas.dtypes}")

## Integración con Pandas

PyArrow puede ser usado como backend en Pandas para mejorar rendimiento y flexibilidad de tipos.

In [ ]:
# Crear DataFrame con Pandas usando PyArrow como backend
df_arrow = pd.DataFrame(
    {
        'id': [1, 2, 3, 4, 5],
        'nombre': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
        'edad': [25, 30, 35, 28, 32]
    }
)

# Convertir columnas a tipos Arrow
df_arrow['nombre'] = df_arrow['nombre'].astype(pd.ArrowDtype(pa.string()))
df_arrow['edad'] = df_arrow['edad'].astype(pd.ArrowDtype(pa.int32()))

print(f"DataFrame:\n{df_arrow}")
print(f"\nDtypes:\n{df_arrow.dtypes}")

In [ ]:
# Comparativa de memoria: NumPy vs PyArrow
import sys

# DataFrame con tipos normales (NumPy)
df_normal = pd.DataFrame({'col': np.arange(1000000)})
mem_normal = df_normal.memory_usage(deep=True).sum() / 1024**2

# DataFrame con tipos Arrow
df_arrow = df_normal.copy()
df_arrow['col'] = df_arrow['col'].astype(pd.ArrowDtype(pa.int64()))
mem_arrow = df_arrow.memory_usage(deep=True).sum() / 1024**2

print(f"Memoria NumPy (normal): {mem_normal:.2f} MB")
print(f"Memoria PyArrow: {mem_arrow:.2f} MB")
print(f"Reducción: {(1 - mem_arrow/mem_normal) * 100:.1f}%")

## Formatos Soportados

PyArrow soporta lectura y escritura de múltiples formatos binarios optimizados.

In [ ]:
# Crear datos de ejemplo
data = {
    'id': list(range(1, 11)),
    'valor': np.random.rand(10),
    'categoría': ['A', 'B', 'A', 'C', 'B'] * 2
}
table = pa.table(data)

# Guardar en Parquet
import pyarrow.parquet as pq
pq.write_table(table, 'tabla_ejemplo.parquet')
print("✓ Guardado en Parquet")

# Leer de Parquet
tabla_leida = pq.read_table('tabla_ejemplo.parquet')
print(f"✓ Leído de Parquet:\n{tabla_leida}")

In [ ]:
# Información de metadatos de Parquet
parquet_file = pq.ParquetFile('tabla_ejemplo.parquet')
print(f"Schema: {parquet_file.schema}")
print(f"Número de filas: {parquet_file.metadata.num_rows}")
print(f"Número de columnas: {parquet_file.metadata.num_columns}")

In [ ]:
import pyarrow.csv as csv

# Leer CSV con PyArrow (más eficiente que Pandas para archivos grandes)
# Primero, crear un CSV de ejemplo
df_ejemplo = pd.DataFrame({
    'id': range(1, 6),
    'nombre': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'valor': [10.5, 20.3, 15.7, 18.2, 22.1]
})
df_ejemplo.to_csv('datos_ejemplo.csv', index=False)

# Leer con PyArrow
tabla_csv = csv.read_csv('datos_ejemplo.csv')
print(f"Tabla desde CSV:\n{tabla_csv}")

## Resumen y Conclusiones

### Cuándo usar PyArrow:

✓ **Trabajar con datos grandes** que requieren optimización de memoria

✓ **Interoperabilidad** entre diferentes sistemas y lenguajes

✓ **I/O de alto rendimiento** con formatos como Parquet

✓ **Backend de Pandas** para mejora de rendimiento

### Características Clave:

* **Almacenamiento columnar eficiente**
* **Tipos de datos ricos y flexibles**
* **Operaciones vectorizadas rápidas**
* **Compresión transparente**
* **Zero-copy en muchas operaciones**

En los próximos notebooks veremos cómo **Polars** aprovecha PyArrow internamente para ofrecer un API aún más potente y eficiente.